In [47]:
import numpy as np
import pyvista as pv
import os
from scipy import interpolate
import matplotlib.pyplot as plt

In [48]:
def create_base_streamlines(mesh):
    w,v,u = mesh.point_data['w'], mesh.point_data['v'], mesh.point_data['u']

    vectors = np.empty((mesh.n_points, 3))
    vectors[:,0] = u
    vectors[:,1] = v
    vectors[:,2] = w

    mesh['vectors'] = vectors

    streamlines, seed_pts = mesh.streamlines(
        'vectors', 
        return_source=True,
        n_points=100,
        source_radius=1.5,
        source_center=(6.0,0,0)
    )
    
    return streamlines, seed_pts

In [49]:
def compute_downsampled_streamlines(s, seeds, base_mesh):
    '''
        Computes streamlines from seed points and down sample size
    '''
    w,v,u = base_mesh.point_data['w'], base_mesh.point_data['v'], base_mesh.point_data['u']

    u.shape = v.shape = w.shape = base_mesh.dimensions


    # Down Sample / Decimate
    u = u[::s, ::s, ::s] 
    v = v[::s, ::s, ::s]
    w = w[::s, ::s, ::s]

    dims = u.shape

    u = u.flatten()
    v = v.flatten()
    w = w.flatten()

    vectors = np.empty((int(len(u)), 3))
    vectors[:,0] = u
    vectors[:,1] = v
    vectors[:,2] = w

    mesh = pv.UniformGrid(
        dims=dims,
        spacing=tuple([x*s for x in list(base_mesh.spacing)]),
        origin=base_mesh.origin
    )

    mesh['vectors'] = vectors
    
    streamlines = mesh.streamlines_from_source(
        vectors='vectors', 
        source=seeds
    )
    
    return streamlines, mesh

In [50]:
def entry_point(filename):
    mesh = pv.read(filename)
    s, seed_pts = create_base_streamlines(mesh)
    s2, m2 = compute_downsampled_streamlines(2, seed_pts, mesh)
    s4, m4 = compute_downsampled_streamlines(4, seed_pts, mesh)
    s8, m8 = compute_downsampled_streamlines(8, seed_pts, mesh)
    s16, m16 = compute_downsampled_streamlines(16, seed_pts, mesh)
    s32, m32 = compute_downsampled_streamlines(32, seed_pts, mesh)
    s64, m64 = compute_downsampled_streamlines(64, seed_pts, mesh)
    return (s, s2, s4, s8, s16, s32, s64), (mesh, m2, m4, m8, m16, m32, m64)

In [51]:
reynolds160 = '.\\Data\\Structured160\\halfcylinder-2.00.vti'
# reynolds640 = '.\\Data\\Structured640\\halfcylinder-0.00.vti'
(ss, ss2, ss4, ss8, ss16, ss32, ss64), (mm, mm2, mm4, mm8, mm16, mm32, mm64) = entry_point(reynolds160)
# k, k2, k4, k8, k16, k32, k64 = entry_point(reynolds640)

In [52]:
p = pv.Plotter()

def plot_streamlines(value):
    error = int(value)
    streamlines = ss
    mesh = mm
    if error >= 2 and error < 4:
        streamlines = ss2
        mesh = mm2
    elif error >= 4 and error < 8:
        streamlines = ss4
        mesh = mm4
    elif error >= 8 and error < 16:
        streamlines = ss8
        mesh = mm8
    elif error >= 16 and error < 32:
        streamlines = ss16
        mesh = mm16
    elif error >= 32 and error < 64:
        streamlines = ss32
        mesh = mm32
    elif error >= 64:
        streamlines = ss64                    
        mesh = mm64
    
    # boundary = mesh.decimate_boundary().extract_all_edges()
    # p.add_mesh(boundary, color='grey', opacity=.75)
    p.add_mesh(streamlines.tube(radius=0.02))
    

In [53]:
p.add_slider_widget(plot_streamlines, [0, 64], title='Downsample Size')
p.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [54]:
def compute_error(stream1, stream2):
    # Distance between each point in streamline
    dv_1 = stream1[1:,:] - stream1[:-1,:]
    dv_2 = stream2[1:,:] - stream2[:-1,:]

    # Length / Magnitude of each delta t
    dl_1 = np.sqrt(np.sum(dv_1**2, axis=1))
    dl_2 = np.sqrt(np.sum(dv_2**2, axis=1))

    t1 = np.r_[0, np.cumsum(dl_1)]
    t2 = np.r_[0, np.cumsum(dl_2)]

    # Lengths of streamline
    length_1 = t1[-1]
    length_2 = t2[-1]

    x1 = interpolate.interp1d(t1, stream1[:,0])
    y1 = interpolate.interp1d(t1, stream1[:,1])
    z1 = interpolate.interp1d(t1, stream1[:,2])
    p1 = lambda t : np.c_[x1(t), y1(t), z1(t)]

    x2 = interpolate.interp1d(t2, stream2[:,0])
    y2 = interpolate.interp1d(t2, stream2[:,1])
    z2 = interpolate.interp1d(t2, stream2[:,2])
    p2 = lambda t : np.c_[x2(t), y2(t), z2(t)]

    t = np.linspace(0, min(length_1, length_2), 11)    
    ind_error = np.linalg.norm(p1(t) - p2(t), axis=1) # Same as magnitude
    tot_error = np.sum(ind_error)
    
    return tot_error

In [55]:
def find_match(stream1, streams2):
         
    for i in range(streams2.n_cells):   
        stream2 = streams2.cell_points(i)
        if(stream1[0][0] == stream2[0][0]):
            return stream2
    return None
            

In [62]:
plo = pv.Plotter()
boundary = mm32.decimate_boundary().extract_all_edges()
plo.add_mesh(boundary, color='grey', opacity=.99)

def plot_streamline(value):
    pp = ss.cell_points(int(value))
    pp2 = find_match(pp, ss2)
    if pp2.any():
        plo.add_mesh(pp2, color='green', name='points2')
    plo.add_mesh(pp, color='red', name='points')
    total_error = compute_error(pp, pp2)
    plo.add_text(f'Total Error : {total_error}', name='error')
    


In [63]:
plo.add_slider_widget(plot_streamline, [0,ss.n_cells], title='Streamline ID', value=0)
plo.show()

Total Error : 0.007435751703077735


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)